In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import sys
import tempfile

from sklearn import svm, metrics
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import requests
from requests.auth import HTTPBasicAuth
from random import randint,random
import json

In [ ]:
#AMBASSADOR_API_IP="localhost:8002"
AMBASSADOR_API_IP="35.203.179.67:31095"

def rest_request(deploymentName,request):
    payload = {'json': json.dumps(request)}
    response = requests.post(
                "http://"+AMBASSADOR_API_IP+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                #"http://localhost:8009/predict",
                #"http://localhost:5300/predict",
                json=request)
                #data=payload)
    print(response)
    return response.json()   
    
def rest_request_auth(deploymentName,request,username,password):
    response = requests.post(
                "https://"+AMBASSADOR_API_IP+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=request,
                auth=HTTPBasicAuth(username, password), verify=False)
    print(response.status_code)
    return response.json()   

In [ ]:
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

In [ ]:
test_data=mnist.test.images
test_labels=np.array(np.where(mnist.test.labels==1))[1]

# Get predictions
**Ensure you have port forwarded the ambassador reverse proxy**

```bash
kubectl port-forward $(kubectl get pod -l service=ambassador -o jsonpath='{.items[0].metadata.name}')  8002:80
```

In [ ]:
test_index = randint(0, test_data.shape[0]-1)
plt.imshow(test_data[test_index].reshape((28,28))); plt.show()
test_samples = test_data[test_index].reshape((1,784)).tolist()
test_samples_with_ids = [{'id': '1', 'values':sample} for sample in test_samples]

request = {"data":{"ndarray":test_samples_with_ids}}

#predictions = rest_request("mnist-svm-1-0",request)
predictions = rest_request_auth("digitsclassifier-1-1-0",request,'admin','admin')

print(json.dumps(predictions, indent=2))
